# Advanced Querying Mongo

Importing libraries and setting up connection

In [1]:
from pymongo import MongoClient
import pandas as pd
client = MongoClient("mongodb://localhost:27017")
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [4]:
client.list_database_names()

['Companies', 'admin', 'config', 'local']

In [5]:
db = client["Companies"]

db.list_collection_names()

['Coleccion']

In [6]:
collection = db["Coleccion"]

In [7]:
query = {'name': 'Babelgum'}
select = {'name': 1, '_id': 0 }
resultado = list(collection.find(query, select))
resultado

[{'name': 'Babelgum'}]

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [21]:

query = {'number_of_employees': {'$gte': 5000}}
select = {'name': 1, '_id': 0, 'number_of_employees': 1}
resultado = list(collection.find(query,select).sort('number_of_employees', -1).limit(20))

resultado

[{'name': 'Siemens', 'number_of_employees': 405000},
 {'name': 'IBM', 'number_of_employees': 388000},
 {'name': 'Toyota', 'number_of_employees': 320000},
 {'name': 'PayPal', 'number_of_employees': 300000},
 {'name': 'Nippon Telegraph and Telephone Corporation',
  'number_of_employees': 227000},
 {'name': 'Samsung Electronics', 'number_of_employees': 221726},
 {'name': 'Accenture', 'number_of_employees': 205000},
 {'name': 'Tata Consultancy Services', 'number_of_employees': 200300},
 {'name': 'Flextronics International', 'number_of_employees': 200000},
 {'name': 'Safeway', 'number_of_employees': 186000},
 {'name': 'Sony', 'number_of_employees': 180500},
 {'name': 'LG', 'number_of_employees': 177000},
 {'name': 'Ford', 'number_of_employees': 171000},
 {'name': 'Boeing', 'number_of_employees': 160000},
 {'name': 'Digital Equipment Corporation', 'number_of_employees': 140000},
 {'name': 'Nokia', 'number_of_employees': 125000},
 {'name': 'MItsubishi Electric', 'number_of_employees': 107000}

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [23]:

query = {'$and': [{'founded_year': {'$lte': 2005}},
                  {'founded_year': {'$gte': 2000}}
                 ]}

select = {'name': 1, '_id': 0, 'founded_year': 1}

resultado = list(collection.find(query, select).limit(5))
resultado

[{'name': 'Wetpaint', 'founded_year': 2005},
 {'name': 'Zoho', 'founded_year': 2005},
 {'name': 'Digg', 'founded_year': 2004},
 {'name': 'Facebook', 'founded_year': 2004},
 {'name': 'Omnidrive', 'founded_year': 2005}]

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [24]:
query = {'$and': [{'founded_year': {'$lt': 2010}}, 
                  {'ipo.valuation_amount': {'$gte': 100000000}}
                 ]}

select = {'name': 1, '_id': 0, 'ipo.valuation_amount': 1}
                
resultado = list(collection.find(query, select).limit(5))
resultado


[{'name': 'Facebook', 'ipo': {'valuation_amount': 104000000000.0}},
 {'name': 'Twitter', 'ipo': {'valuation_amount': 18100000000.0}},
 {'name': 'Yelp', 'ipo': {'valuation_amount': 1300000000}},
 {'name': 'LinkedIn', 'ipo': {'valuation_amount': 9310000000.0}},
 {'name': 'Amazon', 'ipo': {'valuation_amount': 100000000000.0}}]

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [25]:
query = {'$and': [{'founded_year': {'$lt': 2005}},
                  {'number_of_employees': {'$lt': 1000}}
                 ]}

select = {'name': 1, '_id': 0, 'number_of_employees': 1}
                     

resultado = list(collection.find(query, select).sort('number_of_employees', -1).limit(5))
resultado

[{'name': 'Infinera Corporation', 'number_of_employees': 974},
 {'name': 'NorthPoint Communications Group', 'number_of_employees': 948},
 {'name': '888 Holdings', 'number_of_employees': 931},
 {'name': 'Forrester Research', 'number_of_employees': 903},
 {'name': 'Webmetrics', 'number_of_employees': 900}]

### 6. All the companies that don't include the `partners` field.

In [54]:
query = {'$nor': [{'partners': {'$gte': 1}}]}   # primero utilicé la condición $exists para partners pero me devolvía
                                                # una lista vacía, porque todas las compañías tienen esa key aunque
                                                # esté vacía

select = {'name': 1, '_id': 0, 'partners': 1}

resultado = list(collection.find(query, select).limit(5))
resultado

[{'name': 'Wetpaint', 'partners': []},
 {'name': 'AdventNet', 'partners': []},
 {'name': 'Zoho', 'partners': []},
 {'name': 'Digg', 'partners': []},
 {'name': 'Facebook', 'partners': []}]

### 7. All the companies that have a null type of value on the `category_code` field.

In [27]:
query = {'category_code': None}
select = {'name': 1, '_id': 0, 'category_code': 1}

resultado = list(collection.find(query, select).limit(5))
resultado

[{'name': 'Collective', 'category_code': None},
 {'name': 'Snimmer', 'category_code': None},
 {'name': 'KoolIM', 'category_code': None},
 {'name': 'Level9 Media', 'category_code': None},
 {'name': 'VidKing', 'category_code': None}]

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [28]:
query = {'$and': [{'number_of_employees': {'$lt': 1000}},
                  {'number_of_employees': {'$gte': 100}}
                 ]}

select = {'name': 1, '_id': 0, 'number_of_employees': 1}

resultado = list(collection.find(query, select).limit(5))
resultado

[{'name': 'AdventNet', 'number_of_employees': 600},
 {'name': 'AddThis', 'number_of_employees': 120},
 {'name': 'OpenX', 'number_of_employees': 305},
 {'name': 'LifeLock', 'number_of_employees': 644},
 {'name': 'Jajah', 'number_of_employees': 110}]

### 9. Order all the companies by their IPO price in a descending order.

In [29]:
query = {'ipo.valuation_amount': {'$exists': True}}

select = {'name': 1, '_id': 0, 'ipo.valuation_amount': 1}

resultado = list(collection.find(query, select).sort('ipo.valuation_amount', -1).limit(5))
resultado

[{'name': 'GREE', 'ipo': {'valuation_amount': 108960000000.0}},
 {'name': 'Facebook', 'ipo': {'valuation_amount': 104000000000.0}},
 {'name': 'Amazon', 'ipo': {'valuation_amount': 100000000000.0}},
 {'name': 'Twitter', 'ipo': {'valuation_amount': 18100000000.0}},
 {'name': 'Groupon', 'ipo': {'valuation_amount': 12800000000.0}}]

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [30]:
query = {'number_of_employees': {'$exists': True}}

select = {'name': 1, '_id': 0}

resultado = list(collection.find(query, select).sort('number_of_employees', -1).limit(5))
resultado

[{'name': 'Siemens'},
 {'name': 'IBM'},
 {'name': 'Toyota'},
 {'name': 'PayPal'},
 {'name': 'Nippon Telegraph and Telephone Corporation'}]

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [32]:
query = {'$and': [{'founded_month': {'$gte': 6}},
                 {'founded_month': {'$lte': 12}}
                 ]}
select = {'name': 1, '_id': 0, 'founded_month': 1}

resultado = list(collection.find(query, select).limit(5))
resultado

[{'name': 'Wetpaint', 'founded_month': 10},
 {'name': 'Zoho', 'founded_month': 9},
 {'name': 'Digg', 'founded_month': 10},
 {'name': 'Omnidrive', 'founded_month': 11},
 {'name': 'Postini', 'founded_month': 6}]

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [42]:
query = {'$and': [{'founded_year': {'$lte': 2000}},
                  {'acquisition.price_amount': {'$gte': 100000000}}
                 ]}
select = {'name': 1, '_id': 0, 'founded_year': 1}

resultado = list(collection.find(query, select).limit(5))
resultado

[{'name': 'Postini', 'founded_year': 1999},
 {'name': 'SideStep', 'founded_year': 1999},
 {'name': 'PayPal', 'founded_year': 1998},
 {'name': 'Snapfish', 'founded_year': 1999},
 {'name': 'Neopets', 'founded_year': 1999}]

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [44]:
query = {'acquisition.acquired_year': {'$gte': 2010}} 

select = {'name': 1, 'acquisition': 1, '_id': 0}

resultado = list(collection.find(query, select).sort('acquisition.price_amount', -1).limit(5))
pd.DataFrame(resultado)

,name,acquisition
0,T-Mobile,"{'price_amount': 39000000000.0, 'price_currenc..."
1,Goodrich Corporation,"{'price_amount': 18400000000.0, 'price_currenc..."
2,LSI,"{'price_amount': 6600000000.0, 'price_currency..."
3,National Semiconductor,"{'price_amount': 6500000000.0, 'price_currency..."
4,Sybase,"{'price_amount': 5800000000.0, 'price_currency..."


### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [47]:

query = {'founded_year': {'$exists': True}}

select = {'name': 1, 'founded_year': 1, '_id': 0}

resultado = list(collection.find(query, select).sort('founded_year', -1).limit(5))
resultado

[{'name': 'Fixya', 'founded_year': 2013},
 {'name': 'iBazar', 'founded_year': 2013},
 {'name': 'Fluc', 'founded_year': 2013},
 {'name': 'Advaliant', 'founded_year': 2013},
 {'name': 'Wamba', 'founded_year': 2013}]

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [52]:
query = {'acquisition.acquired_day': {'$lte': 7}}

select = {'name': 1, '_id': 0}

resultado = list(collection.find(query, select).sort('acquisition.price_amount', -1).limit(5))
resultado


[{'name': 'National Semiconductor'},
 {'name': 'The Weather Channel'},
 {'name': 'Interactive Data'},
 {'name': 'Macromedia'},
 {'name': 'Qualcomm Atheros'}]

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [56]:
query = {'$and': [{'category_code': 'web'},
                 {'number_of_employees': {'$gte': 4000}}
                ]}

select = {'name': 1, '_id': 0}

resultado = list(collection.find(query, select).sort('number_of_employees').limit(5))
resultado

[{'name': 'Expedia'},
 {'name': 'AOL'},
 {'name': 'Webkinz'},
 {'name': 'Los Angeles Times Media Group'},
 {'name': 'Rakuten'}]

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [64]:
query = {'$and': [{'acquisition.price_amount': {'$gte': 10000000}},
                 {'acquisition.price_currency_code': 'EUR'}
                 ]}

select = {'name': 1, '_id': 0}

resultado = list(collection.find(query, select).limit(5))
resultado

[{'name': 'ZYB'},
 {'name': 'Apertio'},
 {'name': 'Greenfield Online'},
 {'name': 'Webedia'},
 {'name': 'Wayfinder'}]

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [66]:
query = {'$and': [{'acquisition.acquired_month': {'$gte': 1}},
                 {'acquisition.acquired_month': {'$lte': 3}}
                 ]}

select = {'name': 1, 'acquisition': 1, '_id': 0}

resultado = list(collection.find(query, select).limit(5))
pd.DataFrame(resultado)

,name,acquisition
0,Kyte,"{'price_amount': None, 'price_currency_code': ..."
1,NetRatings,"{'price_amount': 327000000, 'price_currency_co..."
2,blogTV,"{'price_amount': None, 'price_currency_code': ..."
3,Livestream,"{'price_amount': None, 'price_currency_code': ..."
4,iContact,"{'price_amount': 169000000, 'price_currency_co..."


# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [70]:
query = {'$and': [{'founded_year': {'$gte': 2000}},
                  {'founded_year': {'$lte': 2010}},
                  {'acquisition.acquired_year': {'$gte': 2011}}
                 ]}

select = {'name': 1, '_id': 0}

resultado = list(collection.find(query, select).limit(5).sort('founded_year', -1))
resultado

[{'name': 'Magento'},
 {'name': 'HyperWeek'},
 {'name': 'Honk'},
 {'name': 'Shutl'},
 {'name': 'Trunkt'}]

### 20. All the companies that have been 'deadpooled' after the third year.

In [71]:
query = {'$expr': {'$gt': ['$deadpooled_year', {'$add':['$founded_year', 3]}]}}
select = {'name': 1, '_id': 0}

resultado = list(collection.find(query, select).limit(5))
resultado

[{'name': 'Babelgum'},
 {'name': 'Thoof'},
 {'name': 'Mercora'},
 {'name': 'Wesabe'},
 {'name': 'Stickam'}]